# Spaceship Titanic: Data Cleaning

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt

## Load and Examine Data

In [2]:
train = pd.read_csv('data/raw/train.csv')
test = pd.read_csv('data/raw/test.csv')

In [3]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [7]:
passenger_id_train = train["PassengerId"].copy()
passenger_id_test  = test["PassengerId"].copy()

In [8]:
y_train = train["Transported"].astype(int)
train = train.drop(columns=["Transported"])

In [9]:
train["Cabin_missing"] = train["Cabin"].isna().astype(int)
test["Cabin_missing"]  = test["Cabin"].isna().astype(int)

In [10]:
def split_cabin(df):
    # Example cabin: "B/0/P" -> Deck="B", CabinNum=0, Side="P"
    # Some cabins are NaN, so fill a placeholder string first
    cabin_str = df["Cabin"].fillna("Unknown/0/Unknown").astype(str)
    parts = cabin_str.str.split("/", expand=True)
    df["Deck"] = parts[0]
    df["CabinNum"] = pd.to_numeric(parts[1], errors="coerce")
    df["Side"] = parts[2]
    return df

In [11]:
train = split_cabin(train)
test  = split_cabin(test)

In [12]:
spend_cols = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
bool_base = ["CryoSleep", "VIP"]
indicator_cols = ["Cabin_missing"]
bool_cols  = bool_base + indicator_cols
cat_cols   = ["HomePlanet", "Destination", "Deck", "Side"]
num_cols   = ["Age", "CabinNum"] + spend_cols

drop_cols  = ["PassengerId", "Name", "Cabin"]

In [13]:
train = train.drop(columns=drop_cols)
test  = test.drop(columns=drop_cols)

In [14]:
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_missing,Deck,CabinNum,Side
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,0,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,0,F,1,S


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   HomePlanet     8492 non-null   object 
 1   CryoSleep      8476 non-null   object 
 2   Destination    8511 non-null   object 
 3   Age            8514 non-null   float64
 4   VIP            8490 non-null   object 
 5   RoomService    8512 non-null   float64
 6   FoodCourt      8510 non-null   float64
 7   ShoppingMall   8485 non-null   float64
 8   Spa            8510 non-null   float64
 9   VRDeck         8505 non-null   float64
 10  Cabin_missing  8693 non-null   int64  
 11  Deck           8693 non-null   object 
 12  CabinNum       8693 non-null   int64  
 13  Side           8693 non-null   object 
dtypes: float64(6), int64(2), object(6)
memory usage: 950.9+ KB


In [16]:
test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_missing,Deck,CabinNum,Side
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,0,G,3,S
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,0,F,4,S
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,0,C,0,S
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,0,C,1,S
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,0,F,5,S


## Handle Missing Values

In [17]:
# Spending NaN -> 0  (interpret missing as "no spending")
train[spend_cols] = train[spend_cols].fillna(0)
test[spend_cols]  = test[spend_cols].fillna(0)

In [18]:
# Numeric NaN (Age, CabinNum) -> median (fit on train only)
age_median = train["Age"].median()
train["Age"] = train["Age"].fillna(age_median)
test["Age"]  = test["Age"].fillna(age_median)

# 3) Fill CabinNum with 0 (neutral value; missingness captured by the indicator)
train["CabinNum"] = train["CabinNum"].fillna(0)
test["CabinNum"]  = test["CabinNum"].fillna(0)

In [19]:
# Booleans -> map to 0/1, fill missing with mode of train
for col in bool_base:
    # Map if still boolean or object
    if train[col].dtype == bool or train[col].dtype == "object":
        train[col] = train[col].map({True: 1, False: 0})
        test[col]  = test[col].map({True: 1, False: 0})
    # Fill any remaining NaNs with 0
    train[col] = train[col].fillna(0)
    test[col]  = test[col].fillna(0)

In [20]:
# Categoricals -> fill NaN with "Unknown"
for col in cat_cols:
    train[col] = train[col].fillna("Unknown")
    test[col]  = test[col].fillna("Unknown")

In [21]:
print(train.isnull().sum())
print(test.isnull().sum())

HomePlanet       0
CryoSleep        0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Cabin_missing    0
Deck             0
CabinNum         0
Side             0
dtype: int64
HomePlanet       0
CryoSleep        0
Destination      0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Cabin_missing    0
Deck             0
CabinNum         0
Side             0
dtype: int64


In [22]:
train.head(25)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_missing,Deck,CabinNum,Side
0,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,0,F,0,S
2,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,0,F,1,S
5,Earth,0.0,PSO J318.5-22,44.0,0.0,0.0,483.0,0.0,291.0,0.0,0,F,0,P
6,Earth,0.0,TRAPPIST-1e,26.0,0.0,42.0,1539.0,3.0,0.0,0.0,0,F,2,S
7,Earth,1.0,TRAPPIST-1e,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0,G,0,S
8,Earth,0.0,TRAPPIST-1e,35.0,0.0,0.0,785.0,17.0,216.0,0.0,0,F,3,S
9,Europa,1.0,55 Cancri e,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0,B,1,P


## Encode/Scale

In [23]:
# One-hot encode
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
ohe.fit(train[cat_cols])

train_cat = ohe.transform(train[cat_cols])
test_cat  = ohe.transform(test[cat_cols])

In [24]:
# Scale numeric
scaler = StandardScaler()
scaler.fit(train[num_cols])

train_num = scaler.transform(train[num_cols])
test_num  = scaler.transform(test[num_cols])

In [25]:
# Booleans as is
train_bool = train[bool_cols].to_numpy()
test_bool  = test[bool_cols].to_numpy()

In [26]:
# 1) Build feature names
num_feature_names  = num_cols                         # ["Age", "CabinNum", ...]
bool_feature_names = bool_cols                        # ["CryoSleep", "VIP", "CabinNum_missing"]
cat_feature_names  = list(ohe.get_feature_names_out(cat_cols))  # e.g. "HomePlanet_Earth", ...

all_feature_names = num_feature_names + bool_feature_names + cat_feature_names

# 2) Combine into NumPy arrays
X_train_clean = np.hstack([train_num, train_bool, train_cat]).astype(np.float32)
X_test_clean  = np.hstack([test_num, test_bool, test_cat]).astype(np.float32)

# 3) Wrap in DataFrames
train_features_df = pd.DataFrame(X_train_clean, columns=all_feature_names)
test_features_df  = pd.DataFrame(X_test_clean,  columns=all_feature_names)

# 4) Add PassengerId (for reference / Kaggle submissions)
train_features_df.insert(0, "PassengerId", passenger_id_train.values)
test_features_df.insert(0, "PassengerId", passenger_id_test.values)

# 5) Add target back to train
train_features_df["Transported"] = y_train.values

In [27]:
print(train_features_df.head(25))

   PassengerId       Age  CabinNum  RoomService  FoodCourt  ShoppingMall  \
0      0001_01  0.711945 -1.141624    -0.333105  -0.281027     -0.283579   
1      0002_01 -0.334037 -1.141624    -0.168073  -0.275387     -0.241771   
2      0003_01  2.036857 -1.141624    -0.268001   1.959998     -0.283579   
3      0003_02  0.293552 -1.141624    -0.333105   0.523010      0.336851   
4      0004_01 -0.891895 -1.139678     0.125652  -0.237159     -0.031059   
5      0005_01  1.060606 -1.141624    -0.333105   0.021662     -0.283579   
6      0006_01 -0.194573 -1.137732    -0.269515   0.683441     -0.278562   
7      0006_02 -0.055109 -1.141624    -0.333105  -0.281027     -0.283579   
8      0007_01  0.433017 -1.135786    -0.333105   0.210921     -0.255149   
9      0008_01 -1.031359 -1.139678    -0.333105  -0.281027     -0.283579   
10     0008_02  0.363284 -1.139678    -0.333105  -0.281027     -0.283579   
11     0008_03  1.130339 -1.139678    -0.274057   4.290638      0.701416   
12     0009_

In [28]:
train_features_df["CabinNum"].value_counts()

CabinNum
-1.141624    217
-0.982044     28
-0.974260     22
-1.104648     22
-1.032643     21
            ... 
 2.057750      1
 1.806704      1
 2.048020      1
 1.343535      1
 2.544273      1
Name: count, Length: 1817, dtype: int64

In [29]:
train["CabinNum"].value_counts()

CabinNum
0       217
82       28
86       22
19       22
56       21
       ... 
1644      1
1515      1
1639      1
1277      1
1894      1
Name: count, Length: 1817, dtype: int64

In [30]:
train_features_df["Cabin_missing"].value_counts()

Cabin_missing
0.0    8494
1.0     199
Name: count, dtype: int64

In [31]:
train_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   Age                        8693 non-null   float32
 2   CabinNum                   8693 non-null   float32
 3   RoomService                8693 non-null   float32
 4   FoodCourt                  8693 non-null   float32
 5   ShoppingMall               8693 non-null   float32
 6   Spa                        8693 non-null   float32
 7   VRDeck                     8693 non-null   float32
 8   CryoSleep                  8693 non-null   float32
 9   VIP                        8693 non-null   float32
 10  Cabin_missing              8693 non-null   float32
 11  HomePlanet_Earth           8693 non-null   float32
 12  HomePlanet_Europa          8693 non-null   float32
 13  HomePlanet_Mars            8693 non-null   float

In [32]:
test_features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   Age                        4277 non-null   float32
 2   CabinNum                   4277 non-null   float32
 3   RoomService                4277 non-null   float32
 4   FoodCourt                  4277 non-null   float32
 5   ShoppingMall               4277 non-null   float32
 6   Spa                        4277 non-null   float32
 7   VRDeck                     4277 non-null   float32
 8   CryoSleep                  4277 non-null   float32
 9   VIP                        4277 non-null   float32
 10  Cabin_missing              4277 non-null   float32
 11  HomePlanet_Earth           4277 non-null   float32
 12  HomePlanet_Europa          4277 non-null   float32
 13  HomePlanet_Mars            4277 non-null   float

In [33]:
# 1. Check feature count matches
print("n_features array:", X_train_clean.shape[1])
print("n_feature names:", len(all_feature_names))
assert X_train_clean.shape[1] == len(all_feature_names)

# 2. Check DataFrame columns (excluding PassengerId + Transported)
feature_cols_df = [c for c in train_features_df.columns
                   if c not in ["PassengerId", "Transported"]]

print("First 10 feature columns in DF:", feature_cols_df[:10])
print("First 10 all_feature_names:", all_feature_names[:10])

assert feature_cols_df == all_feature_names
print("✅ Column names and matrix order line up.")


n_features array: 30
n_feature names: 30
First 10 feature columns in DF: ['Age', 'CabinNum', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'VIP', 'Cabin_missing']
First 10 all_feature_names: ['Age', 'CabinNum', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'VIP', 'Cabin_missing']
✅ Column names and matrix order line up.


In [34]:
train_features_df.columns

Index(['PassengerId', 'Age', 'CabinNum', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'CryoSleep', 'VIP', 'Cabin_missing',
       'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'HomePlanet_Unknown', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e',
       'Destination_Unknown', 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E',
       'Deck_F', 'Deck_G', 'Deck_T', 'Deck_Unknown', 'Side_P', 'Side_S',
       'Side_Unknown', 'Transported'],
      dtype='object')

In [35]:
train_features_df.to_csv('data/clean/train_clean.csv', index=False)
test_features_df.to_csv('data/clean/test_clean.csv', index=False)